In [108]:
import gensim
import multiprocessing as mp
import numpy as np
import pandas as pd
import spacy
import textacy
import textblob


from IPython.core.debugger import Pdb
from gensim.models.word2vec import Word2Vec
from sklearn.metrics.pairwise import cosine_similarity
from string import punctuation
from toolz import curry

In [2]:
pool = mp.Pool(processes=mp.cpu_count())

pdb = Pdb()

ps = punctuation.replace('\'','')

In [3]:
nlp = spacy.load('en_core_web_md')

In [6]:
textacy_preprocess = curry(textacy.preprocess_text)(fix_unicode=True, lowercase=True, no_contractions=True, no_punct=True)

def preprocess(review):
    text = review
    
    for punc in ps:
        text = text.replace(punc, punc + ' ')
    return textacy_preprocess(text)

In [40]:
df = pd.read_csv('./Electronics_5.csv')
text = df.reviewText[df.reviewText.notnull()]
# del df
text[0]

'We got this GPS for my husband who is an (OTR) over the road trucker.  Very Impressed with the shipping time, it arrived a few days earlier than expected...  within a week of use however it started freezing up... could of just been a glitch in that unit.  Worked great when it worked!  Will work great for the normal person as well but does have the "trucker" option. (the big truck routes - tells you when a scale is coming up ect...)  Love the bigger screen, the ease of use, the ease of putting addresses into memory.  Nothing really bad to say about the unit with the exception of it freezing which is probably one in a million and that\'s just my luck.  I contacted the seller and within minutes of my email I received a email back with instructions for an exchange! VERY impressed all the way around!'

In [6]:
# import pickle
# with open('./cleaned_reviews.pickle', 'rb') as infile:
#     cleaned = pickle.load(infile)

In [7]:
# import pickle
# c_splits = []
# with open('./cleaned_split_1.pickle', 'rb') as infile:
#     c_splits.extend(pickle.load(infile))

# with open('./cleaned_split_2.pickle', 'rb') as infile:
#     c_splits.extend(pickle.load(infile))

In [8]:
del df

In [9]:
class ReviewSplitter(object):
    def __init__(self, reviews):
        self.reviews = reviews
        
    def __iter__(self):
        for review in self.reviews:
#             r = preprocess(review)
#             pdb.set_trace()
            yield review.split()

In [ ]:
cleaned = pool.imap(preprocess, text)
pool.close()
pool.join()

In [ ]:
wv = Word2Vec(ReviewSplitter(cleaned), size=200, min_count=500, workers=16)

In [ ]:
wv.save('./w2v_reviews_v2.model')

In [ ]:
pool.terminate()

In [10]:
preprocess('this is a test')

'this is a test'

In [4]:
wv = Word2Vec.load('./w2v_reviews_v2.model').wv

In [29]:
wv.most_similar(positive=['meh', 'sound'])

[('sounding', 0.5803621411323547),
 ('bass', 0.573945164680481),
 ('bassy', 0.5654414892196655),
 ('sounds', 0.5432860851287842),
 ('muddled', 0.5247374773025513),
 ('tinny', 0.5230283737182617),
 ('soundstage', 0.5104070901870728),
 ('vocal', 0.5079558491706848),
 ('booming', 0.49704670906066895),
 ('midrange', 0.4915602505207062)]

In [24]:
wv.wmdistance('tinny', 'quality')

14.82579053321691

In [69]:
wv.most_similar('meh')

[('okay', 0.5531870126724243),
 ('eh', 0.5420130491256714),
 ('alright', 0.5400682687759399),
 ('bad', 0.5200860500335693),
 ('mediocre', 0.5196681618690491),
 ('cheesy', 0.5185725688934326),
 ('crummy', 0.5104964971542358),
 ('lame', 0.5087513327598572),
 ('passable', 0.49705225229263306),
 ('ok', 0.4920235276222229)]

In [104]:
wv.most_similar(positive=['tinny'], negative=['sound'], topn=20)

[('muffled', 0.515436053276062),
 ('hollow', 0.4710240364074707),
 ('muddy', 0.4507274627685547),
 ('dull', 0.4287339150905609),
 ('laggy', 0.42471420764923096),
 ('flimsy', 0.4232005476951599),
 ('flaky', 0.4208517372608185),
 ('silly', 0.4199576675891876),
 ('distorted', 0.4176425337791443),
 ('weak', 0.40768104791641235),
 ('scratchy', 0.40606436133384705),
 ('glitchy', 0.4020988345146179),
 ('disappointing', 0.4006798267364502),
 ('muddled', 0.3994293808937073),
 ('plasticky', 0.3989487290382385),
 ('fragile', 0.3987463414669037),
 ('sloppy', 0.3969285190105438),
 ('cheesy', 0.3945627808570862),
 ('wobbly', 0.3895731270313263),
 ('sluggish', 0.3891417384147644)]

In [96]:
textblob.TextBlob('scratchy').sentiment

Sentiment(polarity=0.0, subjectivity=0.0)

In [113]:
d = wv['sound'] + wv['quality']

/home/cognizac/anaconda2/envs/datainc/lib/python3.6/site-packages/sklearn/utils/validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)
/home/cognizac/anaconda2/envs/datainc/lib/python3.6/site-packages/sklearn/utils/validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)


array([[ 0.61326623]], dtype=float32)